In [65]:
import pickle
import numpy as np
import pandas as pd

from tqdm import tqdm
from dataclasses import dataclass
from scipy.linalg import block_diag

from dynamics import Dynamics, SatelliteDynamics
from utils import get_form_initial_conditions, save_simulation_data

In [66]:
@dataclass
class SimulationConfig:
    # Simulation parameters
    dt: float = 10.0  # Time step [s] (1, 5, 10, 20, 30, 60)
    K: int = (
        6
        * 395  # Simulation duration in timesteps (57000, 11400, 5700, 2850, 1900, 950)
    )
    H: int = 2  # Window size [timesteps]
    seed: int = 42  # Random seed for reproducibility

    # Network parameters
    N: int = 10  # Number of systems
    number_of_chiefs: int = 1  # Number of chiefs
    number_of_deputies: int = N - number_of_chiefs  # Number of deputies
    n_p: int = 3  # Position vector dimension
    n_v: int = 3  # Velocity vector dimension
    n_x: int = n_p + n_v  # State vector dimension
    n: int = N * n_x  # Global state vector dimension
    o_chief: int = 3  # Chief observation vector dimension
    o_deputy: int = 3  # Deputy observation vector dimension
    o: int = (
        number_of_chiefs * o_chief + number_of_deputies * o_deputy
    )  # Global observation vector dimension

    # Observation noise
    r_chief_pos: float = 1e-1  # [m]
    R_chief: np.ndarray = np.diag(np.concatenate([r_chief_pos * np.ones(o_chief)])) ** 2
    r_deputy_pos: float = 1e0  # [m]
    R_deputy: np.ndarray = (
        np.diag(np.concatenate([r_deputy_pos * np.ones(o_deputy)])) ** 2
    )
    R_deputies: np.ndarray = block_diag(R_deputy, R_deputy, R_deputy, R_deputy, R_deputy, R_deputy, R_deputy, R_deputy, R_deputy)
    R: np.ndarray = block_diag(R_chief, R_deputies)

    # Initial deviation noise
    # Warm-start parameters
    p_pos_initial: float = 1e3  # [m]
    p_vel_initial: float = 1e1  # [m / s]
    # Cold-start parameters
    # p_pos_initial: float = 1e2  # [m]
    # p_vel_initial: float = 1e0  # [m / s]
    P_0_spacecraft: np.ndarray = (
        np.diag(
            np.concatenate([p_pos_initial * np.ones(n_p), p_vel_initial * np.ones(n_v)])
        )
        ** 2
    )
    P_0: np.ndarray = block_diag(
        P_0_spacecraft, P_0_spacecraft, P_0_spacecraft, P_0_spacecraft,
        P_0_spacecraft, P_0_spacecraft, P_0_spacecraft, P_0_spacecraft,
        P_0_spacecraft, P_0_spacecraft
    )

    # Levenberg-Marquardt parameters
    lambda_0: float = 1.0
    epsilon: float = 1e-6
    max_iter: int = 100

    # Consensus parameters
    L: int = 1  # Number of consensus iterations
    gamma: float = N  # Consensus gain 1
    pi: float = 1 / N  # Consensus gain 2

    # Newton's method-based algorithms parameters
    grad_norm_order_mag: bool = True
    grad_norm_tol: float = 1e-6
    max_iterations: int = 20

    # Majorization-Minimization parameters
    mm_tol: float = 1e0
    mm_max_iter: int = 20

    # Post-processing parameters
    invalid_rmse: float = 1e2  # [m]
    K_RMSE: int = H  # Index from which the RMSE is calculated

    # Spacecraft parameters
    mass: float = 1.0  # Mass [kg]
    C_drag: float = 2.2  # Drag coefficient
    A_drag: float = 0.01  # Drag area [m^2]
    C_SRP: float = 1.2  # SRP coefficient
    A_SRP: float = 0.01  # SRP area [m^2]

    # Observation model
    @classmethod
    def h(cls, x_vec):
        p_vecs = [x_vec[i : i + cls.n_p] for i in range(0, cls.n, cls.n_x)]
        distances = [p_vecs[i] - p_vecs[j] for (i, j) in [(1, 0), (2, 0), (3, 1), (4, 1), (5, 2), (6, 2), (7, 3), (8, 3), (9, 5)]]
        return np.concatenate((p_vecs[0], np.array(distances).reshape(-1, 1)))

    @classmethod
    def Dh(cls, x_vec):
        first_order_der = np.zeros((cls.o, cls.n))

        first_order_der[: cls.n_p, : cls.n_p] = np.eye(cls.n_p)

        for k, (i, j) in enumerate([(1, 0), (2, 0), (3, 1), (4, 1), (5, 2), (6, 2), (7, 3), (8, 3), (9, 5)], start=1):
            first_order_der[
                cls.n_p * k : cls.n_p * (k + 1), i * cls.n_x : i * cls.n_x + cls.n_p
            ] = np.eye(cls.n_p)
            first_order_der[
                cls.n_p * k : cls.n_p * (k + 1), j * cls.n_x : j * cls.n_x + cls.n_p
            ] = -np.eye(cls.n_p)

        return first_order_der

    @classmethod
    def Hh(cls, x_vec):
        return np.zeros((cls.o * cls.n, cls.n))

In [67]:
config = SimulationConfig()

In [68]:
class HCMCI:
    def __init__(self, W, V):
        self.W = W
        self.V = V

        self.dynamic_model = SatelliteDynamics()
        
        self.h = config.h
        self.Dh = config.Dh

    def prediction(self, dt, x_est_old, Omega_est_old, y):
        x_pred, F = self.dynamic_model.x_new_and_F(dt, x_est_old)
        Omega_pred = (
            self.W
            - self.W
            @ F
            @ np.linalg.inv(Omega_est_old + F.T @ self.W @ F)
            @ F.T
            @ self.W
        )
        q_pred = Omega_pred @ x_pred

        H = self.Dh(x_pred)
        y_est = self.h(x_pred)
        delta_q_est_new = H.T @ self.V @ (y - y_est + H @ x_pred)
        delta_Omega_est_new = H.T @ self.V @ H

        return q_pred, Omega_pred, delta_q_est_new, delta_Omega_est_new

    def init_consensus(self, delta_q, delta_Omega, q, Omega, L):
        delta_q_vec = np.zeros((60, 1, L + 1))
        delta_q_vec[:, :, 0] = delta_q
        delta_Omega_vec = np.zeros((60, 60, L + 1))
        delta_Omega_vec[:, :, 0] = delta_Omega
        q_vec = np.zeros((60, 1, L + 1))
        q_vec[:, :, 0] = q
        Omega_vec = np.zeros((60, 60, L + 1))
        Omega_vec[:, :, 0] = Omega

        return delta_q_vec, delta_Omega_vec, q_vec, Omega_vec

    def correction(self, delta_q_vec, delta_Omega_vec, q_vec, Omega_vec, gamma):
        q_est_new = q_vec[:, :, -1] + gamma * delta_q_vec[:, :, -1]
        Omega_est_new = Omega_vec[:, :, -1] + gamma * delta_Omega_vec[:, :, -1]
        x_est_new = np.linalg.inv(Omega_est_new) @ q_est_new

        return x_est_new, Omega_est_new

In [69]:
formation = 3

# Initial conditions and true state vectors (from Tudat)
with open(f"../test/thesis/7-bigger_formation/data/tudatpy_form{formation}_ts_{int(config.dt)}_K_5700.pkl", "rb") as file:
    X_true = pickle.load(file)
X_initial = X_true[:, :, 0]
    
# Process noise covariance matrix estimation
dynamics_propagator = Dynamics()
K_true = X_true.shape[2]
X_dynamics_propagator = np.zeros((config.n, 1, K_true))
X_dynamics_propagator[:, :, 0] = X_initial
X_tudat_propagator = X_true.transpose(2, 0, 1).reshape(K_true, config.n)
for k, X_k in enumerate(X_tudat_propagator[:-1, :]):
    X_dynamics_propagator[:, :, k + 1] = dynamics_propagator.f(
        config.dt, X_k.reshape(config.n, 1)
    )
X_dynamics_propagator = X_dynamics_propagator.transpose(2, 0, 1).reshape(
    K_true, config.n
)
diff = X_tudat_propagator - X_dynamics_propagator
Q = np.cov(diff.T)
Q_chief = Q[: config.n_x, : config.n_x]
Q_deputies = Q[config.n_x :, config.n_x :]
Q = np.diag(np.diag(Q))
print("Estimated process noise covariance matrix :\n", pd.DataFrame(Q))
X_true = X_true[:, :, : config.K] 

# Set the random seed for reproducibility
if config.seed is not None:
    np.random.seed(config.seed)

Estimated process noise covariance matrix :
           0         1         2         3             4         5         6   \
0   0.000027  0.000000  0.000000  0.000000  0.000000e+00  0.000000  0.000000   
1   0.000000  0.000018  0.000000  0.000000  0.000000e+00  0.000000  0.000000   
2   0.000000  0.000000  0.000026  0.000000  0.000000e+00  0.000000  0.000000   
3   0.000000  0.000000  0.000000  0.000001  0.000000e+00  0.000000  0.000000   
4   0.000000  0.000000  0.000000  0.000000  7.318816e-07  0.000000  0.000000   
5   0.000000  0.000000  0.000000  0.000000  0.000000e+00  0.000001  0.000000   
6   0.000000  0.000000  0.000000  0.000000  0.000000e+00  0.000000  0.000027   
7   0.000000  0.000000  0.000000  0.000000  0.000000e+00  0.000000  0.000000   
8   0.000000  0.000000  0.000000  0.000000  0.000000e+00  0.000000  0.000000   
9   0.000000  0.000000  0.000000  0.000000  0.000000e+00  0.000000  0.000000   
10  0.000000  0.000000  0.000000  0.000000  0.000000e+00  0.000000  0.00000

In [ ]:
# M = 100
M = 1

class Args:
    monte_carlo_sims = M
    algorithm = "hcmci"
    formation = formation
    
args = Args()

X_est_all = []
for m in tqdm(range(M), desc="MC runs", leave=True):
    chief_hcmci = HCMCI(np.linalg.inv(Q), np.linalg.inv(config.R))
    deputy1_hcmci = HCMCI(np.linalg.inv(Q), np.linalg.inv(config.R))
    deputy2_hcmci = HCMCI(np.linalg.inv(Q), np.linalg.inv(config.R))
    deputy3_hcmci = HCMCI(np.linalg.inv(Q), np.linalg.inv(config.R))
    deputy4_hcmci = HCMCI(np.linalg.inv(Q), np.linalg.inv(config.R))
    deputy5_hcmci = HCMCI(np.linalg.inv(Q), np.linalg.inv(config.R))
    deputy6_hcmci = HCMCI(np.linalg.inv(Q), np.linalg.inv(config.R))
    deputy7_hcmci = HCMCI(np.linalg.inv(Q), np.linalg.inv(config.R))
    deputy8_hcmci = HCMCI(np.linalg.inv(Q), np.linalg.inv(config.R))
    deputy9_hcmci = HCMCI(np.linalg.inv(Q), np.linalg.inv(config.R))

    # Generate observations
    Y = np.zeros((config.o, 1, config.K))
    for k in range(config.K):
        Y[:, :, k] = config.h(X_true[:, :, k]) + np.random.multivariate_normal(np.zeros(config.o), config.R).reshape((config.o, 1))

    # Initial state vector and state covariance estimate
    X_est = np.zeros_like(X_true)
    X_est_chief = np.zeros_like(X_true)
    X_est_deputy1 = np.zeros_like(X_true)
    X_est_deputy2 = np.zeros_like(X_true)
    X_est_deputy3 = np.zeros_like(X_true)
    X_est_deputy4 = np.zeros_like(X_true)
    X_est_deputy5 = np.zeros_like(X_true)
    X_est_deputy6 = np.zeros_like(X_true)
    X_est_deputy7 = np.zeros_like(X_true)
    X_est_deputy8 = np.zeros_like(X_true)
    X_est_deputy9 = np.zeros_like(X_true)
    initial_dev = np.random.multivariate_normal(np.zeros(config.n), config.P_0).reshape((config.n, 1))

    X_est[:, :, 0] = X_initial + initial_dev
    X_est_chief[:, :, 0] = X_initial + initial_dev
    X_est_deputy1[:, :, 0] = X_initial + initial_dev
    X_est_deputy2[:, :, 0] = X_initial + initial_dev
    X_est_deputy3[:, :, 0] = X_initial + initial_dev
    X_est_deputy4[:, :, 0] = X_initial + initial_dev
    X_est_deputy5[:, :, 0] = X_initial + initial_dev
    X_est_deputy6[:, :, 0] = X_initial + initial_dev
    X_est_deputy7[:, :, 0] = X_initial + initial_dev
    X_est_deputy8[:, :, 0] = X_initial + initial_dev
    X_est_deputy9[:, :, 0] = X_initial + initial_dev
    Omega_chief = np.linalg.inv(np.diag(initial_dev.reshape(-1) ** 2))
    Omega_deputy1 = np.linalg.inv(np.diag(initial_dev.reshape(-1) ** 2))
    Omega_deputy2 = np.linalg.inv(np.diag(initial_dev.reshape(-1) ** 2))
    Omega_deputy3 = np.linalg.inv(np.diag(initial_dev.reshape(-1) ** 2))
    Omega_deputy4 = np.linalg.inv(np.diag(initial_dev.reshape(-1) ** 2))
    Omega_deputy5 = np.linalg.inv(np.diag(initial_dev.reshape(-1) ** 2))
    Omega_deputy6 = np.linalg.inv(np.diag(initial_dev.reshape(-1) ** 2))
    Omega_deputy7 = np.linalg.inv(np.diag(initial_dev.reshape(-1) ** 2))
    Omega_deputy8 = np.linalg.inv(np.diag(initial_dev.reshape(-1) ** 2))
    Omega_deputy9 = np.linalg.inv(np.diag(initial_dev.reshape(-1) ** 2))

    for t in range(1, config.K):
        q_chief, Omega_chief, delta_q_chief, delta_Omega_chief = chief_hcmci.prediction(config.dt, X_est_chief[:, :, t - 1], Omega_chief, Y[:, :, t])
        q_deputy1, Omega_deputy1, delta_q_deputy1, delta_Omega_deputy1 = deputy1_hcmci.prediction(config.dt, X_est_deputy1[:, :, t - 1], Omega_deputy1, Y[:, :, t])
        q_deputy2, Omega_deputy2, delta_q_deputy2, delta_Omega_deputy2 = deputy2_hcmci.prediction(config.dt, X_est_deputy2[:, :, t - 1], Omega_deputy2, Y[:, :, t])
        q_deputy3, Omega_deputy3, delta_q_deputy3, delta_Omega_deputy3 = deputy3_hcmci.prediction(config.dt, X_est_deputy3[:, :, t - 1], Omega_deputy3, Y[:, :, t])
        q_deputy4, Omega_deputy4, delta_q_deputy4, delta_Omega_deputy4 = deputy4_hcmci.prediction(config.dt, X_est_deputy4[:, :, t - 1], Omega_deputy4, Y[:, :, t])
        q_deputy5, Omega_deputy5, delta_q_deputy5, delta_Omega_deputy5 = deputy5_hcmci.prediction(config.dt, X_est_deputy5[:, :, t - 1], Omega_deputy5, Y[:, :, t])
        q_deputy6, Omega_deputy6, delta_q_deputy6, delta_Omega_deputy6 = deputy6_hcmci.prediction(config.dt, X_est_deputy6[:, :, t - 1], Omega_deputy6, Y[:, :, t])
        q_deputy7, Omega_deputy7, delta_q_deputy7, delta_Omega_deputy7 = deputy7_hcmci.prediction(config.dt, X_est_deputy7[:, :, t - 1], Omega_deputy7, Y[:, :, t])
        q_deputy8, Omega_deputy8, delta_q_deputy8, delta_Omega_deputy8 = deputy8_hcmci.prediction(config.dt, X_est_deputy8[:, :, t - 1], Omega_deputy8, Y[:, :, t])
        q_deputy9, Omega_deputy9, delta_q_deputy9, delta_Omega_deputy9 = deputy9_hcmci.prediction(config.dt, X_est_deputy9[:, :, t - 1], Omega_deputy9, Y[:, :, t])
        
        # Consensus
        delta_q_vec_chief, delta_Omega_vec_chief, q_vec_chief, Omega_vec_chief = chief_hcmci.init_consensus(delta_q_chief, delta_Omega_chief, q_chief, Omega_chief, config.L)
        delta_q_vec_deputy1, delta_Omega_vec_deputy1, q_vec_deputy1, Omega_vec_deputy1 = deputy1_hcmci.init_consensus(delta_q_deputy1, delta_Omega_deputy1, q_deputy1, Omega_deputy1, config.L)
        delta_q_vec_deputy2, delta_Omega_vec_deputy2, q_vec_deputy2, Omega_vec_deputy2 = deputy2_hcmci.init_consensus(delta_q_deputy2, delta_Omega_deputy2, q_deputy2, Omega_deputy2, config.L)
        delta_q_vec_deputy3, delta_Omega_vec_deputy3, q_vec_deputy3, Omega_vec_deputy3 = deputy3_hcmci.init_consensus(delta_q_deputy3, delta_Omega_deputy3, q_deputy3, Omega_deputy3, config.L)
        delta_q_vec_deputy4, delta_Omega_vec_deputy4, q_vec_deputy4, Omega_vec_deputy4 = deputy4_hcmci.init_consensus(delta_q_deputy4, delta_Omega_deputy4, q_deputy4, Omega_deputy4, config.L)
        delta_q_vec_deputy5, delta_Omega_vec_deputy5, q_vec_deputy5, Omega_vec_deputy5 = deputy5_hcmci.init_consensus(delta_q_deputy5, delta_Omega_deputy5, q_deputy5, Omega_deputy5, config.L)
        delta_q_vec_deputy6, delta_Omega_vec_deputy6, q_vec_deputy6, Omega_vec_deputy6 = deputy6_hcmci.init_consensus(delta_q_deputy6, delta_Omega_deputy6, q_deputy6, Omega_deputy6, config.L)
        delta_q_vec_deputy7, delta_Omega_vec_deputy7, q_vec_deputy7, Omega_vec_deputy7 = deputy7_hcmci.init_consensus(delta_q_deputy7, delta_Omega_deputy7, q_deputy7, Omega_deputy7, config.L)
        delta_q_vec_deputy8, delta_Omega_vec_deputy8, q_vec_deputy8, Omega_vec_deputy8 = deputy8_hcmci.init_consensus(delta_q_deputy8, delta_Omega_deputy8, q_deputy8, Omega_deputy8, config.L)
        delta_q_vec_deputy9, delta_Omega_vec_deputy9, q_vec_deputy9, Omega_vec_deputy9 = deputy9_hcmci.init_consensus(delta_q_deputy9, delta_Omega_deputy9, q_deputy9, Omega_deputy9, config.L)

        for l in range(1, config.L + 1):
            delta_q_vec_chief[:, :, l] = config.pi * (delta_q_vec_chief[:, :, l - 1] + delta_q_vec_deputy1[:, :, l - 1] + delta_q_vec_deputy2[:, :, l - 1] + delta_q_vec_deputy3[:, :, l - 1])
            delta_q_vec_deputy1[:, :, l] = config.pi * (delta_q_vec_chief[:, :, l - 1] + delta_q_vec_deputy1[:, :, l - 1] + delta_q_vec_deputy2[:, :, l - 1] + delta_q_vec_deputy3[:, :, l - 1])
            delta_q_vec_deputy2[:, :, l] = config.pi * (delta_q_vec_chief[:, :, l - 1] + delta_q_vec_deputy1[:, :, l - 1] + delta_q_vec_deputy2[:, :, l - 1] + delta_q_vec_deputy3[:, :, l - 1])
            delta_q_vec_deputy3[:, :, l] = config.pi * (delta_q_vec_chief[:, :, l - 1] + delta_q_vec_deputy1[:, :, l - 1] + delta_q_vec_deputy2[:, :, l - 1] + delta_q_vec_deputy3[:, :, l - 1])
            delta_q_vec_deputy4[:, :, l] = config.pi * (delta_q_vec_chief[:, :, l - 1] + delta_q_vec_deputy1[:, :, l - 1] + delta_q_vec_deputy2[:, :, l - 1] + delta_q_vec_deputy3[:, :, l - 1])
            delta_q_vec_deputy5[:, :, l] = config.pi * (delta_q_vec_chief[:, :, l - 1] + delta_q_vec_deputy1[:, :, l - 1] + delta_q_vec_deputy2[:, :, l - 1] + delta_q_vec_deputy3[:, :, l - 1])
            delta_q_vec_deputy6[:, :, l] = config.pi * (delta_q_vec_chief[:, :, l - 1] + delta_q_vec_deputy1[:, :, l - 1] + delta_q_vec_deputy2[:, :, l - 1] + delta_q_vec_deputy3[:, :, l - 1])
            delta_q_vec_deputy7[:, :, l] = config.pi * (delta_q_vec_chief[:, :, l - 1] + delta_q_vec_deputy1[:, :, l - 1] + delta_q_vec_deputy2[:, :, l - 1] + delta_q_vec_deputy3[:, :, l - 1])
            delta_q_vec_deputy8[:, :, l] = config.pi * (delta_q_vec_chief[:, :, l - 1] + delta_q_vec_deputy1[:, :, l - 1] + delta_q_vec_deputy2[:, :, l - 1] + delta_q_vec_deputy3[:, :, l - 1])
            delta_q_vec_deputy9[:, :, l] = config.pi * (delta_q_vec_chief[:, :, l - 1] + delta_q_vec_deputy1[:, :, l - 1] + delta_q_vec_deputy2[:, :, l - 1] + delta_q_vec_deputy3[:, :, l - 1])

            delta_Omega_vec_chief[:, :, l] = config.pi * (delta_Omega_vec_chief[:, :, l - 1] + delta_Omega_vec_deputy1[:, :, l - 1] + delta_Omega_vec_deputy2[:, :, l - 1] + delta_Omega_vec_deputy3[:, :, l - 1])
            delta_Omega_vec_deputy1[:, :, l] = config.pi * (delta_Omega_vec_chief[:, :, l - 1] + delta_Omega_vec_deputy1[:, :, l - 1] + delta_Omega_vec_deputy2[:, :, l - 1] + delta_Omega_vec_deputy3[:, :, l - 1])
            delta_Omega_vec_deputy2[:, :, l] = config.pi * (delta_Omega_vec_chief[:, :, l - 1] + delta_Omega_vec_deputy1[:, :, l - 1] + delta_Omega_vec_deputy2[:, :, l - 1] + delta_Omega_vec_deputy3[:, :, l - 1])
            delta_Omega_vec_deputy3[:, :, l] = config.pi * (delta_Omega_vec_chief[:, :, l - 1] + delta_Omega_vec_deputy1[:, :, l - 1] + delta_Omega_vec_deputy2[:, :, l - 1] + delta_Omega_vec_deputy3[:, :, l - 1])
            delta_Omega_vec_deputy4[:, :, l] = config.pi * (delta_Omega_vec_chief[:, :, l - 1] + delta_Omega_vec_deputy1[:, :, l - 1] + delta_Omega_vec_deputy2[:, :, l - 1] + delta_Omega_vec_deputy3[:, :, l - 1])
            delta_Omega_vec_deputy5[:, :, l] = config.pi * (delta_Omega_vec_chief[:, :, l - 1] + delta_Omega_vec_deputy1[:, :, l - 1] + delta_Omega_vec_deputy2[:, :, l - 1] + delta_Omega_vec_deputy3[:, :, l - 1])
            delta_Omega_vec_deputy6[:, :, l] = config.pi * (delta_Omega_vec_chief[:, :, l - 1] + delta_Omega_vec_deputy1[:, :, l - 1] + delta_Omega_vec_deputy2[:, :, l - 1] + delta_Omega_vec_deputy3[:, :, l - 1])
            delta_Omega_vec_deputy7[:, :, l] = config.pi * (delta_Omega_vec_chief[:, :, l - 1] + delta_Omega_vec_deputy1[:, :, l - 1] + delta_Omega_vec_deputy2[:, :, l - 1] + delta_Omega_vec_deputy3[:, :, l - 1])
            delta_Omega_vec_deputy8[:, :, l] = config.pi * (delta_Omega_vec_chief[:, :, l - 1] + delta_Omega_vec_deputy1[:, :, l - 1] + delta_Omega_vec_deputy2[:, :, l - 1] + delta_Omega_vec_deputy3[:, :, l - 1])
            delta_Omega_vec_deputy9[:, :, l] = config.pi * (delta_Omega_vec_chief[:, :, l - 1] + delta_Omega_vec_deputy1[:, :, l - 1] + delta_Omega_vec_deputy2[:, :, l - 1] + delta_Omega_vec_deputy3[:, :, l - 1])
            
            q_vec_chief[:, :, l] = config.pi * (q_vec_chief[:, :, l - 1] + q_vec_deputy1[:, :, l - 1] + q_vec_deputy2[:, :, l - 1] + q_vec_deputy3[:, :, l - 1])
            q_vec_deputy1[:, :, l] = config.pi * (q_vec_chief[:, :, l - 1] + q_vec_deputy1[:, :, l - 1] + q_vec_deputy2[:, :, l - 1] + q_vec_deputy3[:, :, l - 1])
            q_vec_deputy2[:, :, l] = config.pi * (q_vec_chief[:, :, l - 1] + q_vec_deputy1[:, :, l - 1] + q_vec_deputy2[:, :, l - 1] + q_vec_deputy3[:, :, l - 1])
            q_vec_deputy3[:, :, l] = config.pi * (q_vec_chief[:, :, l - 1] + q_vec_deputy1[:, :, l - 1] + q_vec_deputy2[:, :, l - 1] + q_vec_deputy3[:, :, l - 1])
            q_vec_deputy4[:, :, l] = config.pi * (q_vec_chief[:, :, l - 1] + q_vec_deputy1[:, :, l - 1] + q_vec_deputy2[:, :, l - 1] + q_vec_deputy3[:, :, l - 1])
            q_vec_deputy5[:, :, l] = config.pi * (q_vec_chief[:, :, l - 1] + q_vec_deputy1[:, :, l - 1] + q_vec_deputy2[:, :, l - 1] + q_vec_deputy3[:, :, l - 1])
            q_vec_deputy6[:, :, l] = config.pi * (q_vec_chief[:, :, l - 1] + q_vec_deputy1[:, :, l - 1] + q_vec_deputy2[:, :, l - 1] + q_vec_deputy3[:, :, l - 1])
            q_vec_deputy7[:, :, l] = config.pi * (q_vec_chief[:, :, l - 1] + q_vec_deputy1[:, :, l - 1] + q_vec_deputy2[:, :, l - 1] + q_vec_deputy3[:, :, l - 1])
            q_vec_deputy8[:, :, l] = config.pi * (q_vec_chief[:, :, l - 1] + q_vec_deputy1[:, :, l - 1] + q_vec_deputy2[:, :, l - 1] + q_vec_deputy3[:, :, l - 1])
            q_vec_deputy9[:, :, l] = config.pi * (q_vec_chief[:, :, l - 1] + q_vec_deputy1[:, :, l - 1] + q_vec_deputy2[:, :, l - 1] + q_vec_deputy3[:, :, l - 1])

            Omega_vec_chief[:, :, l] = config.pi * (Omega_vec_chief[:, :, l - 1] + Omega_vec_deputy1[:, :, l - 1] + Omega_vec_deputy2[:, :, l - 1] + Omega_vec_deputy3[:, :, l - 1])
            Omega_vec_deputy1[:, :, l] = config.pi * (Omega_vec_chief[:, :, l - 1] + Omega_vec_deputy1[:, :, l - 1] + Omega_vec_deputy2[:, :, l - 1] + Omega_vec_deputy3[:, :, l - 1])
            Omega_vec_deputy2[:, :, l] = config.pi * (Omega_vec_chief[:, :, l - 1] + Omega_vec_deputy1[:, :, l - 1] + Omega_vec_deputy2[:, :, l - 1] + Omega_vec_deputy3[:, :, l - 1])
            Omega_vec_deputy3[:, :, l] = config.pi * (Omega_vec_chief[:, :, l - 1] + Omega_vec_deputy1[:, :, l - 1] + Omega_vec_deputy2[:, :, l - 1] + Omega_vec_deputy3[:, :, l - 1])
            Omega_vec_deputy4[:, :, l] = config.pi * (Omega_vec_chief[:, :, l - 1] + Omega_vec_deputy1[:, :, l - 1] + Omega_vec_deputy2[:, :, l - 1] + Omega_vec_deputy3[:, :, l - 1])
            Omega_vec_deputy5[:, :, l] = config.pi * (Omega_vec_chief[:, :, l - 1] + Omega_vec_deputy1[:, :, l - 1] + Omega_vec_deputy2[:, :, l - 1] + Omega_vec_deputy3[:, :, l - 1])
            Omega_vec_deputy6[:, :, l] = config.pi * (Omega_vec_chief[:, :, l - 1] + Omega_vec_deputy1[:, :, l - 1] + Omega_vec_deputy2[:, :, l - 1] + Omega_vec_deputy3[:, :, l - 1])
            Omega_vec_deputy7[:, :, l] = config.pi * (Omega_vec_chief[:, :, l - 1] + Omega_vec_deputy1[:, :, l - 1] + Omega_vec_deputy2[:, :, l - 1] + Omega_vec_deputy3[:, :, l - 1])
            Omega_vec_deputy8[:, :, l] = config.pi * (Omega_vec_chief[:, :, l - 1] + Omega_vec_deputy1[:, :, l - 1] + Omega_vec_deputy2[:, :, l - 1] + Omega_vec_deputy3[:, :, l - 1])
            Omega_vec_deputy9[:, :, l] = config.pi * (Omega_vec_chief[:, :, l - 1] + Omega_vec_deputy1[:, :, l - 1] + Omega_vec_deputy2[:, :, l - 1] + Omega_vec_deputy3[:, :, l - 1])

        X_est_chief[:, :, t], Omega_chief = chief_hcmci.correction(delta_q_vec_chief, delta_Omega_vec_chief, q_vec_chief, Omega_vec_chief, config.gamma)
        X_est_deputy1[:, :, t], Omega_deputy1 = deputy1_hcmci.correction(delta_q_vec_deputy1, delta_Omega_vec_deputy1, q_vec_deputy1, Omega_vec_deputy1, config.gamma)
        X_est_deputy2[:, :, t], Omega_deputy2 = deputy2_hcmci.correction(delta_q_vec_deputy2, delta_Omega_vec_deputy2, q_vec_deputy2, Omega_vec_deputy2, config.gamma)
        X_est_deputy3[:, :, t], Omega_deputy3 = deputy3_hcmci.correction(delta_q_vec_deputy3, delta_Omega_vec_deputy3, q_vec_deputy3, Omega_vec_deputy3, config.gamma)
        X_est_deputy4[:, :, t], Omega_deputy4 = deputy4_hcmci.correction(delta_q_vec_deputy4, delta_Omega_vec_deputy4, q_vec_deputy4, Omega_vec_deputy4, config.gamma)
        X_est_deputy5[:, :, t], Omega_deputy5 = deputy5_hcmci.correction(delta_q_vec_deputy5, delta_Omega_vec_deputy5, q_vec_deputy5, Omega_vec_deputy5, config.gamma)
        X_est_deputy6[:, :, t], Omega_deputy6 = deputy6_hcmci.correction(delta_q_vec_deputy6, delta_Omega_vec_deputy6, q_vec_deputy6, Omega_vec_deputy6, config.gamma)
        X_est_deputy7[:, :, t], Omega_deputy7 = deputy7_hcmci.correction(delta_q_vec_deputy7, delta_Omega_vec_deputy7, q_vec_deputy7, Omega_vec_deputy7, config.gamma)
        X_est_deputy8[:, :, t], Omega_deputy8 = deputy8_hcmci.correction(delta_q_vec_deputy8, delta_Omega_vec_deputy8, q_vec_deputy8, Omega_vec_deputy8, config.gamma)
        X_est_deputy9[:, :, t], Omega_deputy9 = deputy9_hcmci.correction(delta_q_vec_deputy9, delta_Omega_vec_deputy9, q_vec_deputy9, Omega_vec_deputy9, config.gamma)
        X_est[:, :, t] = np.concatenate((X_est_chief[:6, :, t], X_est_deputy1[6:12, :, t], X_est_deputy2[12:18, :, t], X_est_deputy3[18:24, :, t], X_est_deputy4[24:30, :, t], X_est_deputy5[30:36, :, t], X_est_deputy6[36:42, :, t], X_est_deputy7[42:48, :, t], X_est_deputy8[48:54, :, t], X_est_deputy9[54:60, :, t]), axis=0)
        # print(f"Position estimation error at time {t}: {np.linalg.norm(X_est[0:6, :, t]   - X_true[0:6, :, t]):.2f} m, {np.linalg.norm(X_est[6:12, :, t]  - X_true[6:12, :, t]):.2f} m, {np.linalg.norm(X_est[12:18, :, t] - X_true[12:18, :, t]):.2f} m, {np.linalg.norm(X_est[18:24, :, t] - X_true[18:24, :, t]):.2f} m, {np.linalg.norm(X_est[24:30, :, t] - X_true[24:30, :, t]):.2f} m, {np.linalg.norm(X_est[30:36, :, t] - X_true[30:36, :, t]):.2f} m, {np.linalg.norm(X_est[36:42, :, t] - X_true[36:42, :, t]):.2f} m, {np.linalg.norm(X_est[42:48, :, t] - X_true[42:48, :, t]):.2f} m, {np.linalg.norm(X_est[48:54, :, t] - X_true[48:54, :, t]):.2f} m, {np.linalg.norm(X_est[54:60, :, t] - X_true[54:60, :, t]):.2f} m")
    X_est_all.append(X_est)

MC runs:   0%|          | 0/1 [00:00<?, ?it/s]

Position estimation error at time 1: 13.63 m, 18.63 m, 20.21 m, 19.69 m, 8.75 m, 21.71 m, 13.35 m, 12.69 m, 16.19 m, 26.42 m
Position estimation error at time 2: 0.08 m, 1.19 m, 2.00 m, 1.16 m, 1.23 m, 2.89 m, 1.45 m, 1.58 m, 2.57 m, 2.75 m
Position estimation error at time 3: 0.16 m, 1.14 m, 1.21 m, 1.50 m, 1.11 m, 1.84 m, 1.89 m, 2.75 m, 2.33 m, 1.86 m
Position estimation error at time 4: 0.16 m, 1.52 m, 2.33 m, 1.75 m, 2.62 m, 2.74 m, 1.51 m, 2.50 m, 3.30 m, 2.68 m
Position estimation error at time 5: 0.26 m, 0.49 m, 0.33 m, 2.26 m, 1.55 m, 1.29 m, 1.96 m, 2.63 m, 1.64 m, 1.45 m
Position estimation error at time 6: 0.39 m, 1.14 m, 1.06 m, 2.20 m, 1.37 m, 2.09 m, 1.84 m, 2.62 m, 1.87 m, 2.64 m
Position estimation error at time 7: 0.15 m, 1.23 m, 1.21 m, 0.58 m, 2.80 m, 2.51 m, 1.75 m, 1.17 m, 0.91 m, 3.54 m
Position estimation error at time 8: 0.15 m, 0.94 m, 1.39 m, 2.45 m, 1.22 m, 3.13 m, 1.09 m, 1.56 m, 3.64 m, 3.30 m
Position estimation error at time 9: 0.08 m, 1.93 m, 1.09 m, 2.

MC runs:   0%|          | 0/1 [00:09<?, ?it/s]


KeyboardInterrupt: 

In [54]:
save_simulation_data(args, X_true, X_est_all)

IndexError: list index out of range